In [ ]:
# create custom project datasets and dataloaders

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch

In [ ]:
!pip install transformers

In [ ]:
!pip install librosa

In [2]:
# imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import json
import os
import torch
import librosa
from PIL import Image
from torchvision import transforms
from transformers import AutoImageProcessor, ResNetForImageClassification, AutoProcessor
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from transformers import BertModel, BertTokenizer, AutoTokenizer
from transformers import Trainer, TrainingArguments, AdamW

In [3]:
data_dir_real = "./drive/MyDrive/Applied Computer Vision Project/acv-presidential-dataset/real"
data_dir_fake = "./drive/MyDrive/Applied Computer Vision Project/acv-presidential-dataset/fake"

In [4]:
# create dataframe containing audio, text, and screenshot images for each data sample
data = []

# real data
presidents = ['biden', 'trump']
for name in presidents:
  for i in range(1,16):
    audio_file = name + '-real-' + str(i) + '-audio.wav'
    audio_path = os.path.join(data_dir_real, 'audio-files')
    text_file = name + '-real-' + str(i) + '.txt'
    text_path = os.path.join(data_dir_real, 'text-files')
    screenshot_folder_name = name + '-real-' + str(i)
    screenshot_folder_path = os.path.join(data_dir_real, 'screenshots', screenshot_folder_name)
    image_files = os.listdir(screenshot_folder_path)
    sample = {'audio': audio_file, 'text': text_file, 'images': image_files, 'audio_path': audio_path, 'text_path': text_path, 'image_path': screenshot_folder_path, 'label': 0}
    data.append(sample)

# deepfake data
for name in presidents:
  for i in range(1,9):
    audio_file = name + '-fake-' + str(i) + '.wav'
    audio_path = os.path.join(data_dir_fake, 'audio-files')
    text_file = name + '-fake-' + str(i) + '.txt'
    text_path = os.path.join(data_dir_fake, 'text-files')
    screenshot_folder_name = name + '-fake-' + str(i)
    screenshot_folder_path = os.path.join(data_dir_fake, 'screenshots', screenshot_folder_name)
    image_files = os.listdir(screenshot_folder_path)
    sample = {'audio': audio_file, 'text': text_file, 'images': image_files, 'audio_path': audio_path, 'text_path': text_path, 'image_path': screenshot_folder_path, 'label': 1}
    data.append(sample)

data_df = pd.DataFrame(data)

In [5]:
data_df

,audio,text,images,audio_path,text_path,image_path,label
0,biden-real-1-audio.wav,biden-real-1.txt,"[biden-real-1-001.jpg, biden-real-1-002.jpg, b...",./drive/MyDrive/Applied Computer Vision Projec...,./drive/MyDrive/Applied Computer Vision Projec...,./drive/MyDrive/Applied Computer Vision Projec...,0
1,biden-real-2-audio.wav,biden-real-2.txt,"[biden-real-2-001.jpg, biden-real-2-002.jpg, b...",./drive/MyDrive/Applied Computer Vision Projec...,./drive/MyDrive/Applied Computer Vision Projec...,./drive/MyDrive/Applied Computer Vision Projec...,0
2,biden-real-3-audio.wav,biden-real-3.txt,"[biden-real-3-001.jpg, biden-real-3-002.jpg, b...",./drive/MyDrive/Applied Computer Vision Projec...,./drive/MyDrive/Applied Computer Vision Projec...,./drive/MyDrive/Applied Computer Vision Projec...,0
3,biden-real-4-audio.wav,biden-real-4.txt,"[biden-real-4-001.jpg, biden-real-4-002.jpg, b...",./drive/MyDrive/Applied Computer Vision Projec...,./drive/MyDrive/Applied Computer Vision Projec...,./drive/MyDrive/Applied Computer Vision Projec...,0
4,biden-real-5-audio.wav,biden-real-5.txt,"[biden-real-5-001.jpg, biden-real-5-002.jpg, b...",./drive/MyDrive/Applied Computer Vision Projec...,./drive/MyDrive/Applied Computer Vision Projec...,./drive/MyDrive/Applied Computer Vision Projec...,0
5,biden-real-6-audio.wav,biden-real-6.txt,"[biden-real-6-001.jpg, biden-real-6-002.jpg, b...",./drive/MyDrive/Applied Computer Vision Projec...,./drive/MyDrive/Applied Computer Vision Projec...,./drive/MyDrive/Applied Computer Vision Projec...,0
6,biden-real-7-audio.wav,biden-real-7.txt,"[biden-real-7-001.jpg, biden-real-7-002.jpg, b...",./drive/MyDrive/Applied Computer Vision Projec...,./drive/MyDrive/Applied Computer Vision Projec...,./drive/MyDrive/Applied Computer Vision Projec...,0
7,biden-real-8-audio.wav,biden-real-8.txt,"[biden-real-8-001.jpg, biden-real-8-002.jpg, b...",./drive/MyDrive/Applied Computer Vision Projec...,./drive/MyDrive/Applied Computer Vision Projec...,./drive/MyDrive/Applied Computer Vision Projec...,0
8,biden-real-9-audio.wav,biden-real-9.txt,"[biden-real-9-001.jpg, biden-real-9-002.jpg, b...",./drive/MyDrive/Applied Computer Vision Projec...,./drive/MyDrive/Applied Computer Vision Projec...,./drive/MyDrive/Applied Computer Vision Projec...,0
9,biden-real-10-audio.wav,biden-real-10.txt,"[biden-real-10-001.jpg, biden-real-10-002.jpg,...",./drive/MyDrive/Applied Computer Vision Projec...,./drive/MyDrive/Applied Computer Vision Projec...,./drive/MyDrive/Applied Computer Vision Projec...,0


In [6]:
data_df.loc[1,'image_path']

'./drive/MyDrive/Applied Computer Vision Project/acv-presidential-dataset/real/screenshots/biden-real-2'

In [7]:
data_df.loc[1,'text_path']

'./drive/MyDrive/Applied Computer Vision Project/acv-presidential-dataset/real/text-files'

In [8]:
data_df.loc[1,'audio_path']

'./drive/MyDrive/Applied Computer Vision Project/acv-presidential-dataset/real/audio-files'

In [9]:
# custom dataset

class CustomDataset(Dataset):
  def __init__(self, data_df):
    self.data_df = data_df
    self.image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")

  def __len__(self):
    return len(self.data_df)

  def __getitem__(self, idx):
    audio = self.data_df.loc[idx, 'audio']
    text = self.data_df.loc[idx, 'text']
    images = self.data_df.loc[idx, 'images']
    label = self.data_df.loc[idx, 'label']

    audio_path = self.data_df.loc[idx, 'audio_path']
    full_audio_path = os.path.join(audio_path, audio)
    opened_audio, sample_rate = librosa.load(full_audio_path, sr=None)

    text_path = self.data_df.loc[idx, 'text_path']
    full_text_path = os.path.join(text_path, text)
    with open(full_text_path, 'r') as file:
      opened_text = file.read()

    image_path = self.data_df.loc[idx, 'image_path']
    processed_images = []

    for img in images:
      full_image_path = os.path.join(image_path, img)
      image = Image.open(full_image_path).convert("RGB")
      processed_image = self.image_processor(images=image, return_tensors="pt")
      processed_image = processed_image["pixel_values"]
      processed_images.append(processed_image)

    item = {
        "audio": opened_audio,
        "sample_rate": sample_rate,
        "text": opened_text,
        "images": processed_images,
        "label": label
    }

    return item



In [10]:
# custom collate function

text_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
audio_processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")

def collate_fn(batch):
  audios = [item['audio'] for item in batch]
  sample_rates = [item['sample_rate'] for item in batch]
  texts = [item['text'] for item in batch]
  image_lists = [item['images'] for item in batch]
  labels = [item['label'] for item in batch]

  processed_audios = []
  for audio, sample_rate in zip(audios, sample_rates):
    inputs = audio_processor(audio, sampling_rate=sample_rate, return_tensors="pt")
    processed_audios.append(inputs.input_values.unsqueeze(0))

  processed_audios = torch.stack(processed_audios, dim=0)

  stacked_images = [torch.stack(images, dim=0) for images in image_lists]
  images = torch.stack(stacked_images, dim=0)

  tokenized_texts = text_tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
  tokenized_texts = tokenized_texts['input_ids']

  text_attention_mask = (tokenized_captions != 0).long()

  labels = torch.LongTensor(labels)

  batch_dict = {
      "audios": processed_audios,
      "texts": tokenized_texts,
      "images": images,
      'text_attention_mask': text_attention_mask,
      "labels": labels
  }

  return batch_dict

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [11]:
# create training, validation, and testing dataframes

train_df, test_df = train_test_split(data_df, test_size=0.25, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [12]:
# create training, validation, and testing datasets using custom dataset class

train_dataset = CustomDataset(train_df)
val_dataset = CustomDataset(val_df)
test_dataset = CustomDataset(test_df)

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

In [13]:
# create training, validation, and testing data loaders

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn, num_workers=0)